# Pybot

In [36]:
import tensorflow as tf
import os
import re
import sys
import string
import numpy as np
from sklearn.metrics.pairwise import cosine_distances
from time import time
import matplotlib.pyplot as plt

import json
import nltk
import tensorflow.keras.preprocessing.text as tftext
from collections import defaultdict
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input, Bidirectional, Concatenate

from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot
import gensim.downloader as api
glove_vectors = api.load("glove-wiki-gigaword-100")
from spellchecker import SpellChecker

In [7]:
from attention import AttentionLayer

In [8]:
import bot_utils

In [11]:
#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

### 0. Hyperparameters

In [12]:
HYPER_PARAMS = {
    'DATA_BATCH_SIZE' : 128,
    'DATA_BUFFER_SIZE' : 10000, 
    'EMBEDDING_DIMENSION' : 100,  # Dimension of the GloVe embedding vector
    'MAX_SENT_LENGTH' : 11,      # Maximum length of sentences
    'MAX_SAMPLES' : 2000000,        # Maximum samples to consider (useful for laptop memory)  200000
    'MIN_WORD_OCCURENCE' : 30,    # Minimum word count. If condition not met word replaced with <UNK>
    'MODEL_LAYER_DIMS' : 500,
    'MODEL_LEARN_RATE' : 1e-3,
    'MODEL_LEARN_EPOCHS' : 10,
    'MODEL_TRAINING' : False # When True, the model will retrain. False: The weights are loaded from file
}


### 1. Importing the data set
https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html

movie_lines:
- lineID
- characterID (who uttered this phrase)
- movieID
- character name
- text of the utterance

movie_conversations:
- characterID of the first character involved in the conversation
- characterID of the second character involved in the conversation
- movieID of the movie in which the conversation occurred
- list of the utterances that make the conversation, in chronological 
order: ['lineID1','lineID2',É,'lineIDN']
has to be matched with movie_lines.txt to reconstruct the actual content


In [15]:
lines = open('movie_lines.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
conversations = open('movie_conversations.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')

### 2. Pre-processing the data
- Perform cleaning
- Extract Q&As
- Tokenize
- Add padding

In [16]:
def clean_text_to_lower(text):
    """Performs cleaning and lowers text"""
    
    text = text.lower()
    # Expand contractions to words, and remove characters
    for word in text.split():
        if word in bot_utils.CONTRACTIONS:
            text = text.replace(word, bot_utils.CONTRACTIONS[word]) 

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text
    
def get_questions_answers(lines, conversations):
    """Builds question and answers from data sets"""
    # Map id to text of the utterance
    id_to_line = {}
    for line in lines:
        line_ = line.split(' +++$+++ ')
        if len(line_) == 5:
            id_to_line[line_[0]] = line_[4]
    
    questions, answers = [], []
    
    for line in conversations:
        # Extract conversation turns
        conversation = line.split(' +++$+++ ')
        if len(conversation) == 4:
            conversation_turns = conversation[3][1:-1].split(', ')
            turns_list = [turn[1:-1] for turn in conversation_turns]   
            # Split to Q & A
            for i_turn in range(len(conversation_turns) - 1):  
                questions.append(clean_text_to_lower(id_to_line[conversation_turns[i_turn][1:-1]]))
                answers.append(clean_text_to_lower(id_to_line[conversation_turns[i_turn + 1][1:-1]]))
                if len(questions) >= HYPER_PARAMS['MAX_SAMPLES']:
                    return questions, answers
    return questions, answers

def tokenize(lines, conversations):
    """Tokenizes sets to sequences of integers, and adds special tokens.
    Also reduces the vocabulary by replacing low frequency words with an unknown token.
    
    Returns:
    tokenizer: tokenizer,  which might be used later to reverse integers to words,
    tokenized_questions: list, questions as sequence of integers
    tokenized_answers: list, answer as sequence of integers
    size_vocab: int, unique number of words in vocabulary.
    special_tokens: dict, mappings for special tokens"""
    
    questions, answers = get_questions_answers(lines, conversations)
    tokenizer = tftext.Tokenizer(oov_token='<UNK>')
    tokenizer.fit_on_texts(questions + answers)
    
    # Make UNK tokens, reindex tokenizer dicts
    sorted_by_word_count = sorted(tokenizer.word_counts.items(), key=lambda kv: kv[1], reverse=True)
    tokenizer.word_index = {}
    tokenizer.index_word = {}
    index = 1
    for word, count in sorted_by_word_count:
        if count >= HYPER_PARAMS['MIN_WORD_OCCURENCE']:
            tokenizer.word_index[word] = index
            tokenizer.index_word[index] = word
            index += 1
    
    # Add special tokens
    special_tokens = {}
    special_tokens['<PAD>'] = 0 
    special_tokens['<UNK>'] = len(tokenizer.word_index)
    special_tokens['<SOS>'] = special_tokens['<UNK>'] + 1
    special_tokens['<EOS>'] = special_tokens['<SOS>'] + 1
    
    for special_token, index_value in special_tokens.items():
        tokenizer.word_index[special_token] = index_value
        tokenizer.index_word[index_value] = special_token
    
    # Tokenize to integer sequences
    tokenized_questions = tokenizer.texts_to_sequences(questions)
    tokenized_answers = tokenizer.texts_to_sequences(answers)
    
    # Size is equal to the last token's index
    size_vocab = special_tokens['<EOS>'] + 1

    # Add sentence position tokens
    tokenized_questions = [[special_tokens['<SOS>']] + tokenized_question + [special_tokens['<EOS>']]
                           for tokenized_question in tokenized_questions]
    
    tokenized_answers   = [[special_tokens['<SOS>']] + tokenized_answer + [special_tokens['<EOS>']]
                           for tokenized_answer in tokenized_answers]
    
    # Add padding at end so we can use a static input size to the model
    tokenized_questions = pad_sequences(tokenized_questions, 
                                        maxlen=HYPER_PARAMS['MAX_SENT_LENGTH'], 
                                        padding='post')
    tokenized_answers   = pad_sequences(tokenized_answers, 
                                        maxlen=HYPER_PARAMS['MAX_SENT_LENGTH'], 
                                        padding='post')
    
    
    return tokenizer, tokenized_questions, tokenized_answers, size_vocab, special_tokens

In [17]:
tokenizer, tokenized_questions, tokenized_answers,\
size_vocab, special_tokens = tokenize(lines, conversations)

In [18]:
size_vocab

6492

In [19]:
len(tokenized_questions)

221616

**Create tf.data.Dataset** <br>
Allows caching and prefetching to speed up training.

In [20]:
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'encoder_inputs': tokenized_questions[:, 1:], # Skip <SOS> token
        'decoder_inputs': tokenized_answers[:, :-1] # Skip <EOS> token
    },
    {
        'outputs': tokenized_answers[:, 1:]  # Skip <SOS> token
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(HYPER_PARAMS['DATA_BUFFER_SIZE'])
dataset = dataset.batch(HYPER_PARAMS['DATA_BATCH_SIZE'])
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

### 3. Create contextual embedding from GloVe
Create embedding layer with out context, from a pre-trained Word2Vec Model from Glove:<br>
https://nlp.stanford.edu/projects/glove/

In [23]:
def load_glove_weights(dimension_embedding):
    """ Load GloVe pre-trained model"""
    path='glove.6B.' + str(dimension_embedding) + 'd.txt'
    word_to_vec = {}
    with open(path, encoding='utf-8') as file:
        for line in file:
            values = line.split()
            word_to_vec[values[0]] = np.asarray(values[1:], dtype='float32')
        file.close()
    return word_to_vec

def load_embedding_weights(word_dictionary, size_vocab, dimension_embedding):
    """Loads embedding weights from GloVe based on our context"""
    word_to_vec = load_glove_weights(dimension_embedding)
    embedding_matrix = np.zeros((size_vocab,
                                HYPER_PARAMS['EMBEDDING_DIMENSION']))
    for word, index in word_dictionary.items():
        embedding_vector = word_to_vec.get(word)
        # Word is within GloVe dictionary
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
    return embedding_matrix, word_to_vec

def build_embedding_layer(word_dictionary, size_vocab, dimension_embedding, length_input): 
    """Builds a non-trainable embedding layer from GloVe 
    pre-trained model, based on our context"""
    embedding_matrix, word_to_vec  = load_embedding_weights(word_dictionary, size_vocab, dimension_embedding)
    size_vocab = size_vocab
    embedding_layer = Embedding(size_vocab,
                                dimension_embedding,
                                input_length=length_input,
                                weights=[embedding_matrix],
                                trainable=False)
    return embedding_matrix, word_to_vec, embedding_layer

In [24]:
embedding_matrix, word_to_vec, embedding_layer = build_embedding_layer(tokenizer.word_index, 
                                                                       size_vocab,
                                                                       HYPER_PARAMS['EMBEDDING_DIMENSION'],
                                                                       HYPER_PARAMS['MAX_SENT_LENGTH'])

### 4. Seq2Seq Model

In [25]:
def seq2seq(embedding_layer, length_input, size_vocab, layer_dims):
    """LSTM Seq2Seq model with attention"""
    length_input = length_input - 1
    # 
    # Encoder,
    # Biderectional (RNNSearch) as explained in https://arxiv.org/pdf/1409.0473.pdf 
    #
    encoder_inputs = Input(shape=(length_input, ), name='encoder_inputs')
    encoder_embedding = embedding_layer(encoder_inputs)
    ecoder_lstm = Bidirectional(LSTM(layer_dims,
                                     return_state=True,
                                     return_sequences=True,
                                     dropout=0.05,
                                     recurrent_initializer='glorot_uniform',
                                     name='encoder_lstm'),
                                name='encoder_bidirectional')
    
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = ecoder_lstm(encoder_embedding)
    # For annotating sequences, we concatenate forward hidden state with backward one as explained in top paper
    state_h = Concatenate(name='encoder_hidden_state')([forward_h, backward_h])
    state_c = Concatenate(name='encoder_cell_state')([forward_c, backward_c])
    encoder_states = [state_h, state_c]
    
    # 
    # Decoder,
    # Unidirectional
    #
    decoder_inputs = Input(shape=(length_input, ), name='decoder_inputs')
    decoder_embedding = embedding_layer(decoder_inputs)
    decoder_lstm = LSTM(layer_dims * 2, # to match bidirectional size
                        return_state=True,
                        return_sequences=True,
                        dropout=0.05,
                        recurrent_initializer='glorot_uniform',
                        name='decoder_lstm')
    # Set encoder to use the encoder state as initial states
    decoder_output,_ , _, = decoder_lstm(decoder_embedding,
                                         initial_state=encoder_states)
    
    # Attention
    attention_layer = AttentionLayer(name='attention_layer')
    attention_output, attention_state = attention_layer([encoder_outputs, decoder_output])
    decoder_concat = Concatenate(axis=-1)([decoder_output, attention_output])
    
    # Output layer
    outputs = Dense(size_vocab, name='outputs', activation='softmax')(decoder_concat)
    
    return Model([encoder_inputs, decoder_inputs], outputs), decoder_embedding

In [31]:
import graphviz

In [32]:
import pydot

**Masked loss** <br>
Since we are dealing with batches of padded sequences, we cannot simply consider all elements of the tensor when calculating loss.

In [34]:
model, decoder_embedding = seq2seq(embedding_layer, 
                                   HYPER_PARAMS['MAX_SENT_LENGTH'], 
                                   size_vocab,
                                   HYPER_PARAMS['MODEL_LAYER_DIMS'])

optimizer = Adam(learning_rate=HYPER_PARAMS['MODEL_LEARN_RATE'])
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['acc'])


In [37]:
# SVG(model_to_dot(model, show_shapes=True, show_layer_names=False, 
#                  rankdir='TB', dpi=65).create(prog='dot', format='svg'))

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


AttributeError: 'NoneType' object has no attribute 'create'

### Training
Optionally load weights

In [38]:
# 200 really
if HYPER_PARAMS['MODEL_TRAINING']:
    model.fit(dataset, epochs=HYPER_PARAMS['MODEL_LEARN_EPOCHS'], batch_size=HYPER_PARAMS['DATA_BATCH_SIZE'])
    model.save('backup_{0}.h5'.format(HYPER_PARAMS['MODEL_LEARN_EPOCHS']))
else:
    model.load_weights('backup_{0}.h5'.format(HYPER_PARAMS['MODEL_LEARN_EPOCHS']))

### 5. Predictions processing
- Follow the same tokeninzing approach as before
- For words that are not in our original context, we use the GloVe embedding to find the most similar word within our vocabulary. If it is still a weird word, it will be replaced by an unknown token

In [39]:
def get_known_words(sentence_words, glove_word_dictionary, local_word_dictionary, topn=20):
    """Pre-process the input text, corrects spelling, 
    get similar words from gensim if input word is out of context
    or replace with <UNK> for bad words."""
    result = []
    for word in sentence_words:
        
        # Correct any spelling mistakes
        spell = SpellChecker()
        word = spell.correction(word)
        
        if word in local_word_dictionary:
            result.append(word) 
        else:
            # Determine if top match is within our local context
            found_sim_word = False
            if word in glove_word_dictionary:
                similar_words = glove_vectors.similar_by_word(word, topn=topn)
                for (sim_word, measure) in similar_words:
                    if sim_word in local_word_dictionary.keys():
                        found_sim_word = True
                        result.append(sim_word)
                        break
            # Funny word
            if not found_sim_word:
                result.append('<UNK>')
    return result
            
def pre_process_new_questions(text, tokenizer, glove_vectors, special_tokens):
    """ Process text to words within our context,
    and tokenize."""
    # Pre-process
    text_ = clean_text_to_lower(text)
    split_text = text_.split(' ')
    split_text = get_known_words(split_text, glove_vectors, tokenizer.word_index)
    processed_question = " ".join(split_text)
    # Tokenize to sequence of ints
    # using original tokenizer
    tokenized_question = tokenizer.texts_to_sequences([processed_question])
    tokenized_question = [[special_tokens['<SOS>']] + 
                          tokenized_question[0] + 
                          [special_tokens['<EOS>']]]

    # Padding                                                                     
    tokenized_question = pad_sequences(tokenized_question, 
                                       maxlen=HYPER_PARAMS['MAX_SENT_LENGTH'], 
                                       padding='post')                                                                       
    return processed_question, tokenized_question[0]

def post_process_new_answers(text_sequence, tokenizer, glove_vectors):
    answer = []
    # Build string from text sequence
    for text_id in text_sequence:
        if text_id != tokenizer.word_index['<EOS>'] and\
           text_id != tokenizer.word_index['<PAD>']:
            answer.append(tokenizer.index_word[text_id])
    answer = " ".join(answer)
    
    # TODO : ADD CBOW/SKIPGRAMS TO FIX <UNK> predictions  (find best word for surrounding words)
    return answer

**Example of correction** <br>
Vanish is not within our model's context, but dissapear is

In [40]:
processed_question, _ = pre_process_new_questions('i wouzld like tso vansish', tokenizer, glove_vectors, special_tokens) 
processed_question

'i would like to disappear'

### 6. Predictions

In [41]:
def get_encoder_decoder(model):
    """ Get the encoder and decoder models,
        used to do inference of text sequences"""
    
    # Encoder model
    encoder_model = Model(model.get_layer('encoder_inputs').output,
                          [model.get_layer('encoder_bidirectional').output[0],
                          [model.get_layer('encoder_hidden_state').output, 
                           model.get_layer('encoder_cell_state').output]])
    
    # Decoder model
    decoder_state_input_h = Input(shape=(HYPER_PARAMS['MODEL_LAYER_DIMS'] * 2, ))
    decoder_state_input_c = Input(shape=(HYPER_PARAMS['MODEL_LAYER_DIMS'] * 2, ))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = model.get_layer('decoder_lstm')(decoder_embedding,
                                                                        initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    
    decoder_model = Model([model.get_layer('decoder_inputs').output, 
                           decoder_states_inputs],
                          [decoder_outputs] + decoder_states)

    return encoder_model, decoder_model

def infer_answer_sentence(model, encoder_model, decoder_model, raw_question, tokenizer, glove_vectors, special_tokens):
    
    processed_question, tokenized_question = pre_process_new_questions(raw_question, 
                                                                       tokenizer, 
                                                                       glove_vectors, 
                                                                       special_tokens)
    tokenized_question = tf.expand_dims(tokenized_question, axis=0)
    
    # Encode input sentence
    encoder_outputs, states = encoder_model.predict(tokenized_question)
    
    # Create starting input with only <SOS> token
    decoder_input = tf.expand_dims(special_tokens['<SOS>'], 0)
    
    decoded_answer_sequence = []
    
    while len(decoded_answer_sequence) < HYPER_PARAMS['MAX_SENT_LENGTH']:
        decoder_outputs , decoder_hidden_states , decoder_cell_states = decoder_model.predict([decoder_input] + states)
        # Apply attention to decoder output
        attention_layer = AttentionLayer()
        attention_outputs, attention_states = attention_layer([encoder_outputs, decoder_outputs])
        decoder_concatenated = Concatenate(axis=-1)([decoder_outputs, attention_outputs])
        dense_output = model.get_layer('outputs')(decoder_concatenated)
        
        # Get argmax of output
        sampled_word = np.argmax(dense_output[0, -1, :])
        
        # Finished output sentence
        if sampled_word == special_tokens['<EOS>']:
            break
        
        # Store prediction, and use it as the decoder's new input
        decoded_answer_sequence.append(sampled_word)
        decoder_input = tf.expand_dims(sampled_word, 0)
        
        # Update internal states each timestep
        states = [decoder_hidden_states, decoder_cell_states]
        
    answer = post_process_new_answers(decoded_answer_sequence, tokenizer, glove_vectors)
    
    return answer

In [42]:
encoder_model, decoder_model = get_encoder_decoder(model)

In [69]:
raw_question = 'do you want LUNCH'
answer = infer_answer_sentence(model, encoder_model, decoder_model, raw_question, tokenizer, glove_vectors, special_tokens)

In [70]:
answer

'no no no no no'

# Telegram link up

In [55]:
import telebot
from telebot import types

from spellchecker import SpellChecker #Pip install
spell = SpellChecker(distance=2)

import warnings
warnings.filterwarnings("ignore")

In [56]:
# API_TOKEN = "1056118906:AAED4TQl8z-w5vd2Eh8tI6mGRdVGZPs_Fu0"

# bot = telebot.TeleBot(API_TOKEN)

# id_=1274801758 #Need to create a ground ID to get this number
# welcome="Please enter /start to proceed /help"
# bot.send_message(1274801758,text=welcome)
# option_dict = []
# option_select=[]

lists to store choices from spelling mistakes

In [82]:
API_TOKEN = "1056118906:AAED4TQl8z-w5vd2Eh8tI6mGRdVGZPs_Fu0"

bot = telebot.TeleBot(API_TOKEN)

'''
Remove this if you dont have a group set on Telegram
'''

#https://api.telegram.org/bot1056118906:AAED4TQl8z-w5vd2Eh8tI6mGRdVGZPs_Fu0/getUpdates
    
    
##########################################################################
id_=1274801758 #Need to create a ground ID to get this number
welcome="Please enter /start to proceed /help"
bot.send_message(1274801758,text=welcome)
####################################################################

option_dict = []
option_select=[]

@bot.message_handler(commands=['help', 'start'])
def send_welcome(message):
    print("send welcome")
    wel = bot.reply_to(message, "Welcome to the NLP purpose built bot of PC and SA. I do not function well, so lets see how this goes")
    bot.register_next_step_handler(wel, check_spelling)

def check_spelling(message):
        try: 
            words=message.text 
            
            test=True
            
            sent=words.split()
            for i in range(len(sent)):
                print("in For")
                test2=False
                word=sent[i].lower()
                check=spell.correction(word)
                print(word)

                if check==word: #correctly spelled
                    continue

                if check != word:
                    test2=True
                    print("In spell check")
                    mistake=True
                    candidate=list(spell.candidates(word))
                    len_candidate=len(candidate)
                    if len_candidate>1:
                        #Closest match
                        correction_one=sent
                        correction_one[i]=candidate[0].upper()
                        A=' '.join(correction_one)
                        #NExt best
                        correction_two=sent
                        correction_two[i]=candidate[1].upper()
                        B=' '.join(correction_two)
                        #Original
                        C=words
                        hold="Did you mean: "+ '\n'+"/A)"+ A + '\n'+"/B)"+B+  '\n'+"/C)"+C
                        option_dict.append(A)
                        option_dict.append(B)
                        option_dict.append(C)

                    else:
                        correction_one=sent
                        correction_one[i]=candidate[0].upper()
                        A=' '.join(correction_one)
                        B=""
                        C=""
                        hold="Did you mean: "+ '\n'+"/A)"+ A + '\n'+"/B)"+words
                        option_dict.append(A)
                        option_dict.append(B)
                        option_dict.append(C)

            
            if test2==True and i==(len(sent)-1):
                print("spell complted with correction needed")
                msg = bot.reply_to(message, hold)
                bot.register_next_step_handler(msg,select_option)
                #print(hold)
            if test2==False and i==(len(sent)-1):
                
                print("sent correct from beg.",message.text)
                #response=get_response(message.text)
               
                raw_question = message.text
                answer = infer_answer_sentence(model, encoder_model, decoder_model, raw_question, tokenizer, glove_vectors, special_tokens)
                response=answer
                
                msg = bot.reply_to(message, response)
                bot.register_next_step_handler(msg, check_spelling)
                
            
            if test==False:
                bot.register_next_step_handler(msg,select_option)
        except Exception as e:
            bot.reply_to(message, 'Its the end of the world part1')
            


def select_option(message):
    print("in select option")
    try:
        option_list=["/A","/B","/C"]
        option = message.text
        if option not in option_list:
            msg = bot.reply_to(message, 'Please select a valid option')
            bot.register_next_step_handler(msg,select_option)
            return
        option_select.append(option)
        
        look_up=option_select[-1]
        look_index=look_up[-1]
        print("look up",look_up)
        print('Look index',look_index)
        
        '''Get response from bot
        need CB stuff to link
        
        '''

        if look_index=="C":
            print("C",option_dict[-1])
            raw_question= (option_dict[-1])
            
        if look_index=="B":
            print("B",option_dict[-2])
            raw_question=(option_dict[-2])
            
        if look_index=="A":
            print("A",option_dict[-3])
            raw_question= (option_dict[-3])
            
            
        #raw_question= 
        print("pulling response",raw_question)
        answer = infer_answer_sentence(model, encoder_model, decoder_model, raw_question, tokenizer, glove_vectors, special_tokens)
        response=answer
        print("anser",answer)

        #response="GET REPLY AGAIN"
        
        msg = bot.reply_to(message, response)
        
        bot.register_next_step_handler(msg, check_spelling)
    except Exception as e:
        bot.reply_to(message, 'Its the end of the world part2')

bot.polling()

send welcome
in For
where
in For
is
in For
the
in For
sea
sent correct from beg. where is the sea
in For
do
in For
you
in For
sleep
sent correct from beg. do you sleep
in For
why
sent correct from beg. why
in For
are
in For
you
in For
human
sent correct from beg. are you human
in For
what
in For
colour
in For
is
in For
the
in For
sky
sent correct from beg. what colour is the sky
in For
go
in For
to
in For
bed
sent correct from beg. go to bed
in For
why
sent correct from beg. why
in For
what
in For
do
in For
you
in For
like
in For
to
in For
eat
sent correct from beg. what do you like to eat
in For
do
in For
you
in For
drink
sent correct from beg. do you drink
in For
are
in For
you
in For
alive
sent correct from beg. are you alive
in For
where
in For
is
in For
batman
sent correct from beg. where is batman
in For
what
in For
is
in For
your
in For
favourite
in For
movie
sent correct from beg. What is your favourite movie
in For
i
in For
looked
in For
for
in For
you
in For
back
in For
at
in